# Reference Star Example

In [ ]:
import sys
sys.path.insert(0, '../')

import numpy as np
import sncosmo

from sn_analysis import reference
from sn_analysis.modeling import get_model_with_pwv, calc_x0_for_z, create_observations_table
from sn_analysis.utils import register_decam_filters


In [ ]:
register_decam_filters()


For context, reference star values are made available by the ``reference`` package.

In [ ]:
reference.get_ref_star_dataframe()


In [ ]:
reference.ref_star_norm_flux('decam_z', 0)


We start by defining a cadence. We won't need the ``gain`` or ``skynoise`` values because we will use a fixed SNR later on. THese values are dropped for clarity.

In [ ]:
obs = create_observations_table(bands=['decam_r', 'decam_i', 'decam_z'])
obs.remove_columns(['gain', 'skynoise'])
obs


We simulate light-curves using the `Model.band_mag` functionality of `sncosmo`. This way we don't need to think through how `sncomso.realize_lc` handles the addition of noise.

In [ ]:
def subtract_ref_from_lc(lc_table, pwv, reference_type='G2'):
    """Subtract reference flux from a light-curve

    Args:
        lc_table     (Table): Astropy table with columns ``flux`` and ``band``
        pwv          (float): PWV value to subtract reference star for
        reference_type (str): Type of reference star (Default 'G2')

    Returns:
        A modified copy of ``lc_table``
    """

    table_copy = lc_table.copy()
    table_copy['ref_flux_nrom'] = np.nan
    
    for band in set(table_copy['band']):
        # The reference flux normalized to the fiducial atm
        norm_ref_flux = reference.ref_star_norm_flux(band, pwv, reference_type)

        table_copy['flux'][table_copy['band'] == band] /= norm_ref_flux
        table_copy['ref_flux_nrom'][table_copy['band'] == band] = norm_ref_flux

    return table_copy


def realize_lc(obs, source, snr=.05, reference_star=False, fid_pwv=4, **params):
    """Simulate a SN light-curve for given parameters

    Light-curves are simulated for the given parameters without any of
    the added effects from ``sncosmo.realize_lc``.

    Args:
        obs       (Table): Observation cadence
        source   (Source): The sncosmo source to use in the simulations
        snr       (float): Signal to noise ratio
        **params         : Values for any model parameters

    Yields:
        Astropy table for each PWV and redshift
    """

    model = get_model_with_pwv(source)
    model.update(params)

    # Set default x0 value according to assumed cosmology and the model redshift
    x0 = params.get('x0', calc_x0_for_z(model['z'], source))
    model.set(x0=x0)

    light_curve = obs[['time', 'band', 'zp', 'zpsys']]
    light_curve['flux'] = model.bandflux(obs['band'], obs['time'], obs['zp'], obs['zpsys'])

    if reference_star:
        # Use recursion to create a light-curve with the same parameters
        # but with PWV at the fiducial value
        fiducial_params = params.copy()
        fiducial_params['pwv'] = fid_pwv
        fiducial_lc = realize_lc(obs, source, snr, **fiducial_params)
        
        # Normalize the SN light-curve
        light_curve['flux'] /= fiducial_lc['flux']
        light_curve['zp'] = -2.5 * np.log10(fiducial_lc['flux'])
        
        # Subtract the reference star
        light_curve = subtract_ref_from_lc(light_curve, model['pwv'], reference_star)
        

    light_curve['fluxerr'] = light_curve['flux'] * snr
    return light_curve


In [ ]:
realize_lc(obs, 'salt2-extended', z=.1, pwv=10)


In [ ]:
realize_lc(obs, 'salt2-extended', reference_star='G2', z=.1, pwv=10)
